<a href="https://colab.research.google.com/github/OlenaVN/Notebooks-for-data-science-bootcamp-practice/blob/main/Create__R%D0%B5_Ranking_full_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cross-Encoder Re-Ranking**

Install sentence-transformers python library  and import 'CrossEncoder' from this library.

In [7]:
# place your code here
!pip install sentence_transformers

In [8]:
from sentence_transformers import CrossEncoder

From typing  import List, Tuple

In [9]:
from typing import List, Tuple

Load the pre-trained cross-encoder model 'cross-encoder/ms-marco-MiniLM-L-6-v2' and save the model object to the 'model' variable. To load, use CrossEncoder() with two arguments: the first is the model name as string and the second argument is max_length=512

Load the model object into the 'model' variable.

In [10]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Use function:

In [12]:
def rerank_documents(
    query: str,  documents: List[str],  top_k: int = 10
) -> List[Tuple[int, str, float]]:
    """
    Re-rank documents based on relevance to query.
    Args:
        query: The search query
        documents: List of document texts from initial retrieval
        top_k: Number of top results to return
    Returns:
        List of (original_index, document, score) tuples, sorted by relevance
    """
    # Create query-document pairs for the cross-encoder
    # Each pair will be scored independently
    pairs = [[query, doc] for doc in documents]

    # Get relevance scores for all pairs
    # The model outputs a single score per pair
    scores = model.predict(pairs)

    # Combine with original indices and sort by score descending
    scored_docs = [
        (idx, doc, float(score))
        for idx, (doc, score) in enumerate(zip(documents, scores))
    ]
    scored_docs.sort(key=lambda x: x[2], reverse=True)

    return scored_docs[:top_k]

Simulate retrieval results: use the following data.

In [23]:
retrieved_docs = [
        "Kubernetes pod monitoring requires metrics collection from the kubelet.",
        "Docker containers can be monitored using cAdvisor metrics.",
        "To check pod status, use kubectl get pods command.",
        "Prometheus is commonly used for Kubernetes monitoring.",
        "Pod resource limits should be set in the deployment spec.",
        "OneUptime provides real-time Kubernetes monitoring dashboards.",
        "Container orchestration platforms need observability solutions.",
        "The kubectl logs command shows container output.",
    ]
query = "How do I monitor Kubernetes pods?"

In [24]:
results = rerank_documents(query, retrieved_docs, top_k=5)

In [25]:
print("Re-ranked results:")
for idx, doc, score in results:
  print(f"  [{score:.3f}] (orig #{idx}): {doc[:60]}...")

Re-ranked results:
  [6.196] (orig #0): Kubernetes pod monitoring requires metrics collection from t...
  [2.877] (orig #5): OneUptime provides real-time Kubernetes monitoring dashboard...
  [2.824] (orig #3): Prometheus is commonly used for Kubernetes monitoring....
  [0.142] (orig #2): To check pod status, use kubectl get pods command....
  [-8.542] (orig #7): The kubectl logs command shows container output....
